In [19]:
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

#engine = create_engine('sqlite:///C:/Users/8135t/Desktop/Project2-WaterWorld/waterdb.sqlite', echo=False)

In [82]:
from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy


In [24]:
engine = create_engine('sqlite:///C:/Users/8135t/Desktop/Project2-WaterWorld/tao-branch/db/waterdb.sqlite', echo=False)

In [25]:
inspector = inspect(engine)
inspector.get_table_names()

['Aquastat_table', 'WQI_table']

In [26]:
columns = inspector.get_columns('Aquastat_table')
for column in columns:
    print(column["name"], column["type"])

index INTEGER
country VARCHAR(30)
Variable VARCHAR(40)
Year INTEGER
Value FLOAT
unit VARCHAR(10)
iso VARCHAR(20)


In [27]:
another_columns = inspector.get_columns('WQI_table')
for column in another_columns:
    print(column["name"], column["type"])

index INTEGER
iso VARCHAR(20)
country VARCHAR(30)
H2O_current FLOAT


In [28]:
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [29]:
Base.classes.keys()

['Aquastat_table', 'WQI_table']

In [30]:
WQI_table = Base.classes.WQI_table
Aquastat_table = Base.classes.Aquastat_table

In [7]:
session = Session(engine)

In [31]:
results = session.query(WQITable.iso, WQITable.country, WQITable.H2O_current).limit(10).all()

In [32]:
for result in results:
    print(result)

('AFG', 'Afghanistan', 25.75)
('ALB', 'Albania', 66.56)
('DZA', 'Algeria', 60.26)
('AGO', 'Angola', 9.2)
('ATG', 'Antigua and Barbuda', 53.33)
('ARG', 'Argentina', 73.07)
('ARM', 'Armenia', 61.94)
('AUS', 'Australia', 98.44)
('AUT', 'Austria', 94.63)
('AZE', 'Azerbaijan', 48.53)


In [35]:
aqua_results = session.query(Aquastat_table.country, Aquastat_table.Variable,
                             Aquastat_table.Year, Aquastat_table.Value,
                             Aquastat_table.unit, Aquastat_table.iso).limit(30).all()

In [37]:
# for result in aqua_results:
#     print(result)

In [38]:
stmt = session.query(Aquastat_table).statement

In [41]:
import pandas as pd

In [42]:
aqua_df = pd.read_sql_query(stmt, session.bind)


In [49]:
stmt_wqi = session.query(WQI_table).statement

In [50]:
wqi_df = pd.read_sql_query(stmt_wqi, session.bind)

In [86]:
list(aqua_df.country.unique())[0]

'Afghanistan'

In [89]:
sel = [
        Aquastat_table.country,
        Aquastat_table.Variable,
        Aquastat_table.Year,
        Aquastat_table.Value,
        Aquastat_table.unit,
        Aquastat_table.iso,
    ]

In [101]:
afg_stmt = session.query(*sel).filter(Aquastat_table.iso == 'AFG').statement


In [103]:
afg_df = pd.read_sql_query(afg_stmt,session.bind)


In [164]:
afg_df.head()

,country,Variable,Year,Value,unit,iso
0,Afghanistan,Total population,1962,9344.0,E,AFG
1,Afghanistan,Total population,1967,10369.0,E,AFG
2,Afghanistan,Total population,1972,11717.0,E,AFG
3,Afghanistan,Total population,1977,13056.0,E,AFG
4,Afghanistan,Total population,1982,12667.0,E,AFG


In [176]:
sample_dict={}
sample_dict['country']=afg_df['country'].iloc[0]
sample_dict['iso']=afg_df['iso'].iloc[0]

for stuff, stuff2 in afg_df.groupby(['country','Variable'],as_index=True):
#     print(f'{stuff[0]} and {stuff[1]}')
    data={}
    data['Year'] = list(stuff2.Year)
    data['Value'] = list(stuff2.Value)
#     print('----------------------------')
#     print(list(stuff2.Year))
#     print(list(stuff2.Value))
#     print('############################')
    sample_dict[stuff[1]]=data

In [177]:
sample_dict

{'country': 'Afghanistan',
 'iso': 'AFG',
 'Population density': {'Year': [1962,
   1967,
   1972,
   1977,
   1982,
   1987,
   1992,
   1997,
   2002,
   2007,
   2012,
   2015],
  'Value': [14.31,
   15.88,
   17.95,
   20.0,
   19.4,
   17.37,
   21.06,
   27.62,
   32.91,
   39.64,
   45.53,
   49.82]},
 'Rural population with access to safe drinking-water (JMP)': {'Year': [1992,
   1997,
   2002,
   2007,
   2012,
   2015],
  'Value': [16.2, 19.5, 27.6, 35.6, 43.7, 47.0]},
 'Total population': {'Year': [1962,
   1967,
   1972,
   1977,
   1982,
   1987,
   1992,
   1997,
   2002,
   2007,
   2012,
   2015],
  'Value': [9344.0,
   10369.0,
   11717.0,
   13056.0,
   12667.0,
   11338.0,
   13746.0,
   18034.0,
   21487.0,
   25878.0,
   29727.0,
   32527.0]},
 'Total population with access to safe drinking-water (JMP)': {'Year': [1992,
   1997,
   2002,
   2007,
   2012,
   2015],
  'Value': [21.3, 25.0, 33.8, 42.6, 51.6, 55.3]},
 'Urban population': {'Year': [1962,
   1967,
   19

In [146]:
afg_df.groupby(['country','Variable'])['Value'].count()

country      Variable                                                 
Afghanistan  Population density                                           12
             Rural population with access to safe drinking-water (JMP)     6
             Total population                                             12
             Total population with access to safe drinking-water (JMP)     6
             Urban population                                             12
             Urban population with access to safe drinking-water (JMP)     6
Name: Value, dtype: int64

In [100]:
afg_meta_data = {}
for result in afg_results:
    print(result)

('Afghanistan', 'Total population', 1962, 9344.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1967, 10369.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1972, 11717.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1977, 13056.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1982, 12667.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1987, 11338.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1992, 13746.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1997, 18034.0, 'E', 'AFG')
('Afghanistan', 'Total population', 2002, 21487.0, 'E', 'AFG')
('Afghanistan', 'Total population', 2007, 25878.0, 'E', 'AFG')
('Afghanistan', 'Total population', 2012, 29727.0, 'E', 'AFG')
('Afghanistan', 'Total population', 2015, 32527.0, 'E', 'AFG')
('Afghanistan', 'Urban population', 1962, 804.9, 'E', 'AFG')
('Afghanistan', 'Urban population', 1967, 1066.0, 'E', 'AFG')
('Afghanistan', 'Urban population', 1972, 1434.0, 'E', 'AFG')
('Afghanistan', 'Urban population', 1977, 1888.0, 'E', 'AFG'

In [99]:
for result in afg_results:
    if result[1]=='Total population':
        print(result)

('Afghanistan', 'Total population', 1962, 9344.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1967, 10369.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1972, 11717.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1977, 13056.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1982, 12667.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1987, 11338.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1992, 13746.0, 'E', 'AFG')
('Afghanistan', 'Total population', 1997, 18034.0, 'E', 'AFG')
('Afghanistan', 'Total population', 2002, 21487.0, 'E', 'AFG')
('Afghanistan', 'Total population', 2007, 25878.0, 'E', 'AFG')
('Afghanistan', 'Total population', 2012, 29727.0, 'E', 'AFG')
('Afghanistan', 'Total population', 2015, 32527.0, 'E', 'AFG')


In [80]:
for isso in (set(aqua_df.iso)-set(wqi_df.iso)):
    print(aqua_df.loc[aqua_df['iso']==isso]['country'].iloc[0])
    print(aqua_df.loc[aqua_df['iso']==isso]['iso'].iloc[0])
    print('######################################################')

Yemen
YEM
######################################################
Tuvalu
TUV
######################################################
San Marino
SMR
######################################################
Saint Kitts and Nevis
KNA
######################################################
Syrian Arab Republic
SYR
######################################################
South Sudan
SSD
######################################################
Somalia
SOM
######################################################
Faroe Islands
FRO
######################################################
Democratic People's Republic of Korea
PRK
######################################################
Cook Islands
COK
######################################################
Holy See
VAT
######################################################
Monaco
MCO
######################################################
Liechtenstein
LIE
######################################################
Andorra
AND
#######################################

In [45]:
engine.execute('SELECT * FROM WQITable LIMIT 10').fetchall()

[(0, 'AFG', 'Afghanistan', 25.75),
 (1, 'ALB', 'Albania', 66.56),
 (2, 'DZA', 'Algeria', 60.26),
 (3, 'AGO', 'Angola', 9.2),
 (4, 'ATG', 'Antigua and Barbuda', 53.33),
 (5, 'ARG', 'Argentina', 73.07),
 (6, 'ARM', 'Armenia', 61.94),
 (7, 'AUS', 'Australia', 98.44),
 (8, 'AUT', 'Austria', 94.63),
 (9, 'AZE', 'Azerbaijan', 48.53)]

In [33]:
engine.execute('SELECT * FROM AquastatTable LIMIT 10').fetchall()

[(0, 'Afghanistan', 'Total population', 1962, 9344.0, 'E', 'AFG'),
 (1, 'Afghanistan', 'Total population', 1967, 10369.0, 'E', 'AFG'),
 (2, 'Afghanistan', 'Total population', 1972, 11717.0, 'E', 'AFG'),
 (3, 'Afghanistan', 'Total population', 1977, 13056.0, 'E', 'AFG'),
 (4, 'Afghanistan', 'Total population', 1982, 12667.0, 'E', 'AFG'),
 (5, 'Afghanistan', 'Total population', 1987, 11338.0, 'E', 'AFG'),
 (6, 'Afghanistan', 'Total population', 1992, 13746.0, 'E', 'AFG'),
 (7, 'Afghanistan', 'Total population', 1997, 18034.0, 'E', 'AFG'),
 (8, 'Afghanistan', 'Total population', 2002, 21487.0, 'E', 'AFG'),
 (9, 'Afghanistan', 'Total population', 2007, 25878.0, 'E', 'AFG')]